## Multivariate time series prediction using Long Short Term Memory (LSTM)

### Theory of Recurrent Neural Network (RNN)

The most important difference between RNN and Fully Connected Network(FCN) is that RNN does sequential processing of input data and maintains a hidden state.
The hidden state is updated for each sequence input. When we train the model, it learns how to update the hidden state in the best possible way such that it can be used to predict the next value in the sequence. Basic example:

### Let's consider a network that learns to count the number of 1's in a binary sequence.

So, in an FCN, we feed the entire binary sequence at once, e.g. 1, 0, 1, 1 is fed at once, and it multiplies with a weight matrix of a 4x4 matrix and the computes flows forward.

But, in a LSTM, 
- Model first takes 1 and an initial hidden state h which is normally assigned to be 0, updates the hidden state h,
- Then takes 0 and h and updates hidden state h,
- Then takes 1 and h updates hidden state h,
- Also finally takes 1 and h and updates the hidden state h.

  This final h value is used to predict the number of 1's in the binary sequence. Normally this h is passed through a fully connected layer to get the required final output.

Mathematically,

$$ h_t = W_{xh}x_t + b_t + W_{hh} h_{t-1} + b_h $$

Final output is calculated as
$$ y_t = h_t W_{hy}^T + b_y $$

Here is a PyTorch implementation of an LSTM neural network which has 1 LSTM layer and 1 fully connected layer.


### LSTM

LSTM is basically an RNN where the way we calculate the hidden state is more advanced and involves multiple equations. In practice, LSTM has been implemented in different time series modeling applications. Pytorch implementation of LSTM uses the following equations in each layer.

We initialize the $h_{t-1}$ and $c_{t-1}$ to zeros in every forward pass
$$\begin{array}{ll} \\
         Input Gate:   i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
         Forget Gate:   f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
         Candidate Cell State:   g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
         Output Gate:   o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
         Cell State Update:   c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
         Hidden State:   h_t = o_t \odot \tanh(c_t) \\
        \end{array}$$



In [1]:
import torch
import numpy as np

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import optuna

/home/kpanthi/.conda/envs/cenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = 'final_data.csv'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Implement determinism. Set a fixed value for random seed so that when the parameters are initialized, they are initialized same across all experiments.
torch.manual_seed(42)

# If you are using CUDA, also set the seed for it
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Set the seed for NumPy
np.random.seed(42)

Using device: cuda


Here we define **RiverData** a custom Dataset class to load the dataset we have. It extends the Pytorch Dataset class.  
- We need to define \_\_init__() function which can be used for loading data from the file and optionally for data preprocessing.
- Thereafter we define \_\_len__() function which gives the length of dataset.
- Then we define \_\_getitem__() function which returns an instance of (feature, label) tuple which can be used for model training.
  For our time series data, feature means the past values to be used for training and label means the future values to be predicted.

In [4]:
class RiverData(torch.utils.data.Dataset):
    
    def __init__(self, df, target, datecol, seq_len, pred_len):
        self.df = df
        self.datecol = datecol
        self.target = target
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.setIndex()
        

    def setIndex(self):
        self.df.set_index(self.datecol, inplace=True)
    

    def __len__(self):
        return len(self.df) - self.seq_len - self.pred_len


    def __getitem__(self, idx):
        if len(self.df) <= (idx + self.seq_len+self.pred_len):
            raise IndexError(f"Index {idx} is out of bounds for dataset of size {len(self.df)}")
        df_piece = self.df[idx:idx+self.seq_len].values
        feature = torch.tensor(df_piece, dtype=torch.float32)
        label_piece = self.df[self.target][idx + self.seq_len:  idx+self.seq_len+self.pred_len].values
        label = torch.tensor(label_piece, dtype=torch.float32)
        return (feature, label) 

### Normalize the data

In [5]:
df = pd.read_csv(path)
raw_df = df.drop('DATE', axis=1, inplace=False)
scaler = MinMaxScaler()

# Apply the transformations
df_scaled = scaler.fit_transform(raw_df)

df_scaled = pd.DataFrame(df_scaled, columns=raw_df.columns)
df_scaled['DATE'] = df['DATE']
df = df_scaled

Some advanced Python syntax has been used here. \
*common_args : it's used to pass arguments to a function, where common_args represents a python list \
**common_args: it's used to pass arguments to a function, where common_args represents a python dictionary

In [6]:

train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

seq_len = 13
pred_len = 1
num_features = 7

common_args = ['gauge_height', 'DATE', seq_len, pred_len]
train_dataset = RiverData(df[:train_size], *common_args)
val_dataset = RiverData(df[train_size: train_size+val_size], *common_args)
test_dataset = RiverData(df[train_size+val_size : len(df)], *common_args)


In [7]:
# Important parameters

BATCH_SIZE = 512 # keep as big as can be handled by GPU and memory
SHUFFLE = False # we don't shuffle the time series data
DATA_LOAD_WORKERS = 1 # it depends on the amount of data you need to load
learning_rate = 1e-3


In [8]:
from torch.utils.data import DataLoader

common_args = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE}
train_loader = DataLoader(train_dataset, **common_args)
val_loader = DataLoader(val_dataset, **common_args)
test_loader = DataLoader(test_dataset, **common_args)

### Here we define our PyTorch model.

BasicLSTMNetwork is the model class, it extends the Module class provided by pytorch. \
- We define \_\_init__() function. It sets up layers and defines the model parameters.
- Also, we define forward() function which defines how the forwared pass computation occurs

In [9]:
class BasicLSTMNetwork(torch.nn.Module):
    
    def __init__(self, seq_len, pred_len):
        # call the constructor of the base class
        super().__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_features = num_features
        self.n_layers = 1
        
        self.n_hidden = 128
        
        # define layers for combining across time series
        self.lstm1 = torch.nn.LSTM(input_size = self.num_features, hidden_size = self.n_hidden, num_layers=self.n_layers, batch_first = True)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(self.n_hidden * self.seq_len, self.pred_len)


    def init_hidden(self, batchsize):
        device = next(self.parameters()).device
        hidden_state = torch.zeros(self.n_layers, batchsize, self.n_hidden, device=device)
        cell_state = torch.zeros(self.n_layers, batchsize, self.n_hidden, device=device)
        return hidden_state, cell_state

    
    def forward(self, x):
        batchsize, seqlen, featlen = x.size()
        self.hidden_states = self.init_hidden(batchsize)
        lstm_out, self.hidden_states = self.lstm1(x, self.hidden_states)
        lstm_out = lstm_out.contiguous().view(batchsize, -1)
        lstm_out = self.relu(lstm_out)
        lstm_out = self.fc1(lstm_out)
        return lstm_out
# Note that the gradients are stored insize the FC layer objects
# For each training example we need to get rid of these gradients

In [10]:
loss = torch.nn.MSELoss()


In [11]:
for i, (f,l) in enumerate(train_loader):
    print('features shape: ', f.shape)
    print('labels shape: ', l.shape)
    break

features shape:  torch.Size([512, 13, 7])
labels shape:  torch.Size([512, 1])


In [16]:
# define metrics
import numpy as np
epsilon = np.finfo(float).eps

def wape_function(y, y_pred):
    """Weighted Average Percentage Error metric in the interval [0; 100]"""
    y = np.array(y)
    y_pred = np.array(y_pred)
    nominator = np.sum(np.abs(np.subtract(y, y_pred)))
    denominator = np.add(np.sum(np.abs(y)), epsilon)
    wape = np.divide(nominator, denominator) * 100.0
    return wape

def nse_function(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    return (1-(np.sum((y_pred-y)**2)/np.sum((y-np.mean(y))**2)))


def evaluate_model(model, data_loader):
    # following line prepares the model for evaluation mode. It disables dropout and batch normalization if they have 
    # are part of the model. For our simple model, it's not necessary. Still, I'm going to use it.

    model.eval()
    all_inputs = torch.empty((0, seq_len, num_features))
    all_labels = torch.empty(0, pred_len)
    for inputs, labels in data_loader:
        all_inputs = torch.vstack((all_inputs, inputs))
        all_labels = torch.vstack((all_labels, labels))
    
    with torch.no_grad():
        all_inputs = all_inputs.to(device)
        outputs = model(all_inputs).detach().cpu()
        avg_val_loss = loss(outputs, all_labels)
        nse = nse_function(all_labels.numpy(), outputs.numpy())
        wape = wape_function(all_labels.numpy(), outputs.numpy())
        
    print(f'NSE : {nse}', end=' ')
    print(f'WAPE : {wape}', end=' ')
    print(f'Validation Loss: {avg_val_loss}')
    model.train()
    return avg_val_loss


In [17]:
def objective(trial):
    # Here we define the search space of the hyper-parameters. Optuna uses byaesian optimization to find the optimal values of the hyperparameters.
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)

    model = BasicLSTMNetwork(seq_len, pred_len)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

    # I have kept it low for the demonstration purpose. In real world scenario it should be kept high ~ 50.
    num_epochs = 10
    
    best_val_loss = float('inf')

    # Same for this, I have kept it low for demonstration purpose. In real world scenario it should be high ~ 5 to 10.
    patience = 1
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = []
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss_val = loss(outputs, labels)
    
            # calculate gradients for back propagation
            loss_val.backward()
    
            # update the weights based on the gradients
            optimizer.step()
    
            # reset the gradients, avoid gradient accumulation
            optimizer.zero_grad()
            epoch_loss.append(loss_val.item())
    
        avg_train_loss = sum(epoch_loss)/len(epoch_loss)
        print(f'Epoch {epoch+1}: Traning Loss: {avg_train_loss}', end=' ')
        avg_val_loss = evaluate_model(model, val_loader)
    
        # Check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model_trial.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                # Load the best model before stopping
                model.load_state_dict(torch.load('best_model_trial.pth'))
                break

        # Report intermediate objective value
        trial.report(best_val_loss, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value (Best Validation Loss):', trial.value)
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')




[I 2024-11-19 16:04:24,044] A new study created in memory with name: no-name-757d3e92-2f76-4796-9da6-0c87a6cc172f
/local_scratch/slurm.1125506/ipykernel_3252748/2556302494.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/local_scratch/slurm.1125506/ipykernel_3252748/2556302494.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)


Epoch 1: Traning Loss: 0.014093939006564369 NSE : -0.6168206930160522 WAPE : 77.44650288799536 Validation Loss: 0.027603207156062126
Epoch 2: Traning Loss: 0.01087123653708885 NSE : 0.0015901923179626465 WAPE : 60.48224610044913 Validation Loss: 0.01704537309706211
Epoch 3: Traning Loss: 0.007364156993320519 NSE : 0.5486051440238953 WAPE : 39.16127082360514 Validation Loss: 0.007706447504460812
Epoch 4: Traning Loss: 0.004622035719534657 NSE : 0.7580589205026627 WAPE : 27.21856395390195 Validation Loss: 0.004130543675273657
Epoch 5: Traning Loss: 0.0033482389509362324 NSE : 0.797127902507782 WAPE : 24.704020972741354 Validation Loss: 0.0034635381307452917
Epoch 6: Traning Loss: 0.002861688583516067 NSE : 0.8124763369560242 WAPE : 23.735947389957904 Validation Loss: 0.003201501676812768
Epoch 7: Traning Loss: 0.0026119323752789173 NSE : 0.8206541240215302 WAPE : 23.199395989556592 Validation Loss: 0.003061886178329587
Epoch 8: Traning Loss: 0.0024772764033703537 NSE : 0.8248678296804428

[I 2024-11-19 16:05:43,663] Trial 0 finished with value: 0.002907732268795371 and parameters: {'lr': 0.0007561625800237071, 'weight_decay': 0.004708536115006036}. Best is trial 0 with value: 0.002907732268795371.


NSE : 0.8296834826469421 WAPE : 22.595315115826185 Validation Loss: 0.002907732268795371
Epoch 1: Traning Loss: 0.006959353710053758 NSE : 0.7103018164634705 WAPE : 28.061775580116954 Validation Loss: 0.004945878405123949
Epoch 2: Traning Loss: 0.002870339009707538 NSE : 0.7805216610431671 WAPE : 21.699795769673123 Validation Loss: 0.003747048554942012
Epoch 3: Traning Loss: 0.00239318782798807 NSE : 0.872596874833107 WAPE : 18.43789683084679 Validation Loss: 0.0021750922314822674
Epoch 4: Traning Loss: 0.0021330302906097656 

[I 2024-11-19 16:06:15,384] Trial 1 finished with value: 0.0021750922314822674 and parameters: {'lr': 0.00739110623107165, 'weight_decay': 0.0032251005173378213}. Best is trial 1 with value: 0.0021750922314822674.


...

NSE : 0.5608006417751312 WAPE : 35.934107239321534 Validation Loss: 0.007498240098357201
Epoch 1: Traning Loss: 0.011952241111799678 

[I 2024-11-19 16:14:33,331] Trial 18 pruned. 


NSE : -0.2190399169921875 WAPE : 67.2187743644896 Validation Loss: 0.020812084898352623
Epoch 1: Traning Loss: 0.010832582341213379 

[I 2024-11-19 16:14:41,063] Trial 19 pruned. 


NSE : -0.021014690399169922 WAPE : 61.451041594035395 Validation Loss: 0.017431294545531273
Number of finished trials: 20
Best trial:
  Value (Best Validation Loss): 0.0006196001777425408
  Params:
    lr: 0.0010292516182143097
    weight_decay: 0.0002503903940662566


In [18]:
import optuna.visualization as vis

# Optimization history
fig1 = vis.plot_optimization_history(study)
fig1.write_html("optimization_history_lstm.html")